## Autocomplete and Language Models

A **language model** (LM) estimates the probabilities of sentences in a corpus. They treat sentences as word sequences and compute the probability of the sequence. Similarly, they can estimate the probability of the next word in an incomplete sentence given a history of words. So, they can be used for autocomplete, word suggestion, speech recognition, spelling correction and so on.

### N-Gram Overview
**N-gram** is a sequence of N words. In a corpus of "I am happy, because I am learning" 1-grams (or unigrams) are {I, am, happy, because, learning} and 2-grams (or bigrams) are {I am, am happy, happy because, because I, am learning}. The word order matters. In order to represent sequences the following notation is used:

$$
w_1^m = w_1 w_2 ... w_m
$$


We can estimate word probabilities using n-grams. For instance, for the corpus "I am happy, because I am learning", $P(w_i=I) = 2 / 7 $ since the word "I" occurs twice in the corpus of 7 words. In other words, unigram probability of a word is equal to its normalized frequency. So the probability of **unigram** is:

$$
P(w) = \frac{C(w)}{m}
$$
For bigrams, we adopt a conditional probability approach and compute $P(w_{i+1} | w_i)$. We calculate how many times the bigram $w_i w_{i+1}$ occurs and count of $w_i$. For instance, $P(am learning) = P(am | learning) = 1/2$, since "am" is followed by "learning" once and occurs twice in the corpus. So the probability of **bigram** is:

$$
P(y|x) = \frac{C(x   y)}{\sum_w C(x w)} = \frac{C(x   y)}{C(x)}
$$

We can generalize this approach for all $N$  to calculate the probability of N-gram as:
$$
P(w_N | w_1^{N-1}) = \frac{C(w_1^{N-1}w_N)}{C(w_1)^{N-1}}
$$
where $C(w)$ is the count of word $w$.

### Sequence Probabilities
We can use bi-gram language model to compute probability of sentences. We simply multiply the probability of each bi-gram in the sentence and obtain the joint probability. For the sentence "The teacher drinks tea", we compute $P(the)P(teacher|the)P(drinks|teacher)P(tea|drinks)$ using the above formula. With this approach, we asumme that the probability of a word depends only on the immediate predecessor, which is a *Markovian assumption*. 

We can generalize bi-gram approach to n-gram to consider n-1 previous words. Yet, it gets less and less likely for n-grams to be found on the corpus as n gets larger. Thus, most of the probabilities are computed as 0, leading to inaccurate probability estimates.

We can modify this estimation approach by adding a start symbol $(<s>)$ to the beginning of each sentence. This way, we swap the unigram term in the beginning with a bigram term: $P(the|<s>)$. This approach applies to n-grams as well, by adding n-1 start token, instead of one. Similarly, we can add a end token to each sentence $</s>$ to predict if the sentence ends or continues. To apply these modifications, we add these tokens to each sentence in the corpus and treat them as regular words.

In short, we can construct a bi-gram language models with the following steps:

- Add start and end tokens to each setence in the corpus.
- Count bigrams in the corpus. A count matrix $C$ can be used to store this information where $C_{ij}$ is equal to count of $"w_j w_i"$ in the corpus. We can use smoothing here to assign non-zero probability to non-existent bigrams as well.
- Divide each element with the row sum to convert counts to probabilties.
- Multiply bigram probabilities to estimate probabilty of a sentence. Entire sentence modeled with bigram: 
$$
P(w_1^n) \approx \prod_{i=1}^n P(w_i | w_{i-1})
$$
- Use logarithm to overcome the risk of underflow.

### Language Model Evaluation

We evaluate language models using **perplexity** which is the inverse probability of the test set to the power of $m$ (number of words in the corpus including $</s>$, but not $<s>$). One can think perplexity as a measure of the complexity in a sample of text. In other words, perplexity measures how likely that test set occurs based on the language model, i.e. whether a set of sentences written by humans rather than by a simple programs choosing random words. Since it is the inverse probability, better language models obtain lower perplexity in the test sets. We can compute perplexity of a bigram language model on test set $W$ as follows:

$$
PP(W) = \sqrt[m]{\prod_{i=1}^m \prod_{j=1}^{|s_i|} \frac{1}{P(w_j^{i}| w_{j-1}^{i}) }}
$$
where $w_j^i$ is the $j^{th}$ word in the $i^{th}$ sentence.

Alternatively, we can concatenate the sentences in the test set and compute perplexity as:

$$
PP(W) = \sqrt[m]{\prod_{i=1}^{m} \frac{1}{P(w_i| w_{i-1})}}
$$

To overcome underflows, we can compute log perplexity as well by taking $\log_2$ of the formula: 

$$\log P P(W)=-\frac{1}{m} \sum_{i=1}^{m} \log _{2}\left(P\left(w_{i} \mid w_{i-1}\right)\right)$$

### Out of Vocabulary Words

During test time, the language model can encounter words not in the vocabulary. Such words are called *unknown* or *out of vocabulary (OOV)* words. To handle such cases, we add some OOV words to training corpus as well. One method is to add a constraint that a word must occur at least $n$ times to be counted in the vocabulary. With this constraint, words more rare than $n$ is replaced with $</UNK>$ token and treated as the same word. During test time, if a word is not in the vocabulary, it is also replaced with $</UNK>$.

**Remark:** Perplexities of two models are comparable only if they have the same vocabulary.

### Smoothing
When we train an n-gram, the probabilities of some words may be skewed. To deal with the n-grams that do not occur in the corpus we use smoothing techniques. Some commonly used techniques are listed below:

- Laplacian or add-one smoothing: It adds 1 both to the numerator and to each bigram in the denominator sum.

$$
P(w_N | w_1^{N-1}) = \frac{C(w_{N-1}, w_N) + 1}{C(w_{N-1}) + V}
$$

- Add-k smoothing: It adds k both to the numerator and to each bigram in the denominator sum.

$$
P(w_N | w_1^{N-1}) = \frac{C(w_{N-1}, w_N) + k}{C(w_{N-1}) + k*V}
$$

- Backoff: If the higher order n-gram probability is missing, use the lower order n-gram probability multiplied by a constant. 

- Interpolation: Use the linear interpolation of all orders of n-gram, i.e. combine the weighted probabilities of n-grams. 